## Familiarization of TNG Data Acess
### Goals

* Graph 3D time-dependent distribution of things (e.g. stellar mass, Europeum, star formation rate)

In [1]:
import requests
import h5py
import numpy as np
import matplotlib.pyplot as plt

In [2]:
base_url = 'http://www.tng-project.org/api/'
headers = {'api-key': '869b608c817d0a731e5439be3345bdf7'}

In [3]:
def get(path, params=None):
    # make HTTP GET request to path
    r = requests.get(path, params=params, headers=headers)

    # raise exception if response code is not HTTP SUCCESS (200)
    r.raise_for_status()

    if r.headers['content-type'] == 'application/json':
        return r.json() # parse json responses automatically

    if 'content-disposition' in r.headers:
        filename = r.headers['content-disposition'].split("filename=")[1]
        with open(filename, 'wb') as f:
            f.write(r.content)
        return filename # return the filename string

    return r

In [6]:
r = get(base_url)
name_list = [sim['name'] for sim in r['simulations']]
index = name_list.index('Illustris-3')
sim = get(r['simulations'][index]['url'])

In [ ]:
#var inits
total_star_masses, total_sfr = 0, 0
snap_vars = []

#Getting all snapshots
snaps_json = get(sim['snapshots'])
for i in range(0, len(snaps_json)):

    #Getting snap
    snap = get(snaps_json[i]['url'])
    snap_num = snap['number']
    snap_file = get(snap['files']['snapshot'])

    #Getting all chuncks
    for j in range(0, len(snap_file['files'])):
        file = snap_file['files'][j]
        cutout = get(file, {'gas':'StarFormationRate', 'stars':'Masses'})
        
        with h5py.File(cutout,'r') as f:
            
            try:
                star_masses = sum(f['PartType4']['Masses'][:])
            except KeyError:
                star_masses = 0
                
            try:
                star_formationRate = f['PartType0']['StarFormationRate'][:]
            except KeyError:
                star_formationRate = 0
        
        total_star_masses += star_masses
        snap_vars.append((snap_num, total_star_masses))
        
snap_vars

In [86]:

print(star_masses, star_formationRate)

31.18910099961795 [0. 0. 0. ... 0. 0. 0.]


There are a lot of variables and sometimes it is hard to remember what each one of them mean, this link http://www.tng-project.org/data/docs/specifications/#sec1a gives a description of all of these variables and a basic structure of the API.